In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/result1.csv')
df.head()

,_id,topic,heading,summary,date_published,author,content,tags
0,5ef0d2b4d83b3587cb500ec9,"[""analysis""]","[""Don’t discriminate against non-resident migr...","[""Create a legal regime that allows them to ac...","[""Updated: Jun 22, 2020 19:30 IST""]","[""KP Krishnan and Anirudh Burman""]","[""Prime Minister (PM) Narendra Modi recently a...","[""India"",""Narendra Modi"",""BIHAR"",""Haryana"",""Mi..."
1,5ef0d2b8d83b3587cb500ecb,"[""opinion""]","[""India has closed military gap with China alo...","[""Over the past dozen years India has not only...","[""Updated: Jun 22, 2020 16:01 IST""]","[""Pramit Pal Chaudhuri""]","[""India has the "","", says a recent Harvard Uni...","[""Indian Army"",""Galwan Valley"",""India China bo..."
2,5ef0d2bad83b3587cb500ecc,"[""opinion""]","[""Ladakh faceoff: Looking back for a way forwa...","[""DRAW THE LINE Policing the borders and preve...","[""Updated: Jun 22, 2020 18:06 IST""]","[""Lt Gen GS Sihota (retd)""]","[""On December 11, 1917, General Edmund Allenby...",NaN
3,5ef0d2bdd83b3587cb500ecd,"[""editorials""]","[""Redesign the model village scheme | HT Edito...","[""For MPs to take it up, expand its ambit to a...","[""Updated: Jun 22, 2020 18:56 IST""]",NaN,"[""A study commissioned by the Union ministry o...",NaN
4,5ef0d2bed83b3587cb500ece,"[""columns""]","[""How China has overplayed its hand, writes Ba...","[""Beijing’s belligerence reflects a familiar h...","[""Updated: Jun 22, 2020 19:31 IST""]","[""Baijayant “Jay” Panda""]","[""Almost 2,500 years ago, Greek historian, Thu...","[""CHINA"",""India""]"


In [7]:
len(df.content.unique()),  len(df.content)

(154, 162)

In [8]:
df.content[0]

'["Prime Minister (PM) Narendra Modi recently announced that India must become ","(self-reliant). One aspect of this could be that India will remove barriers within its internal markets to truly become a single market. It will remove the hurdles to efficiency improvements and become more competitive. The Goods and Services Tax (GST) was a step in this direction. Recent decisions to remove hurdles in inter-state agricultural trade are also similar. For agricultural and industrial products, as well as capital, India is increasingly becoming a single market. The creation of a barrier-free domestic market is also an intent reflected in Article 301 of the Constitution.","However, there is one market where frictions are being added rather than reduced. This is the labour market. For different reasons, leaders from out-migration and in-migration states have made statements suggesting that there may be more impediments to the inter-state migration of workers. Some states have announced prefere

In [14]:
# time range for which these articles are published
#min(df.date_published).split("T")[0], max(df.date_published).split("T")[0]

In [17]:
# the number of articles in each section
Counter(df.topic)

Counter({'["analysis"]': 11,
         '["art-and-culture"]': 1,
         '["bollywood"]': 7,
         '["business"]': 9,
         '["cities"]': 1,
         '["columns"]': 4,
         '["cricket"]': 4,
         '["delhi"]': 8,
         '["editorials"]': 9,
         '["education"]': 13,
         '["fashion-and-trends"]': 4,
         '["fitness"]': 1,
         '["football"]': 3,
         '["health"]': 1,
         '["hollywood"]': 1,
         '["india"]': 17,
         '["it-s-viral"]': 12,
         '["more-lifestyle"]': 4,
         '["mumbai"]': 10,
         '["music"]': 1,
         '["opinion"]': 2,
         '["other-sports"]': 4,
         '["regional-movies"]': 1,
         '["tennis"]': 1,
         '["travel"]': 1,
         '["tv"]': 12,
         '["world"]': 11,
         nan: 9})

In [25]:
df.isnull().sum()

_id                   0
topic                 9
heading               0
summary               0
date_published        9
author               20
content               7
tags                 27
processed_heading     0
dtype: int64

In [29]:
df.dropna(subset=['content'], inplace = True)
df.dropna(subset = ['topic'], inplace = True)

In [30]:
df.shape

(148, 10)

In [20]:
import pandas as pd
import numpy as np
import re

import gensim
from gensim import corpora,models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
# data preprocessing
stemmer = SnowballStemmer(language='english',ignore_stopwords=True)
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [24]:
# preview after preprocessing
doc_sample = df['content'][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['["Prime', 'Minister', '(PM)', 'Narendra', 'Modi', 'recently', 'announced', 'that', 'India', 'must', 'become', '","(self-reliant).', 'One', 'aspect', 'of', 'this', 'could', 'be', 'that', 'India', 'will', 'remove', 'barriers', 'within', 'its', 'internal', 'markets', 'to', 'truly', 'become', 'a', 'single', 'market.', 'It', 'will', 'remove', 'the', 'hurdles', 'to', 'efficiency', 'improvements', 'and', 'become', 'more', 'competitive.', 'The', 'Goods', 'and', 'Services', 'Tax', '(GST)', 'was', 'a', 'step', 'in', 'this', 'direction.', 'Recent', 'decisions', 'to', 'remove', 'hurdles', 'in', 'inter-state', 'agricultural', 'trade', 'are', 'also', 'similar.', 'For', 'agricultural', 'and', 'industrial', 'products,', 'as', 'well', 'as', 'capital,', 'India', 'is', 'increasingly', 'becoming', 'a', 'single', 'market.', 'The', 'creation', 'of', 'a', 'barrier-free', 'domestic', 'market', 'is', 'also', 'an', 'intent', 'reflected', 'in', 'Article', '301', 'of', 'the', 'Constitution."

In [36]:
# Preprocess the headline text, saving the results as ‘processed_docs’
df['processed_heading'] = df['heading'].map(preprocess)
df['processed_content'] = df['content'].map(preprocess)
df['processed_topic'] = df['topic'].map(preprocess)
df['processed_heading'] = df['processed_heading'].apply(lambda x: ' '.join(x))
df['processed_content'] = df['processed_content'].apply(lambda x: ' '.join(x))
df['processed_topic'] = df['processed_topic'].apply(lambda x: ' '.join(x))
df.head() 

,_id,topic,heading,summary,date_published,author,content,tags,processed_heading,processed_content,processed_topic
0,5ef0d2b4d83b3587cb500ec9,"[""analysis""]","[""Don’t discriminate against non-resident migr...","[""Create a legal regime that allows them to ac...","[""Updated: Jun 22, 2020 19:30 IST""]","[""KP Krishnan and Anirudh Burman""]","[""Prime Minister (PM) Narendra Modi recently a...","[""India"",""Narendra Modi"",""BIHAR"",""Haryana"",""Mi...",discrimin resid migrant opinion,prime minist narendra modi recent announc indi...,analysi
1,5ef0d2b8d83b3587cb500ecb,"[""opinion""]","[""India has closed military gap with China alo...","[""Over the past dozen years India has not only...","[""Updated: Jun 22, 2020 16:01 IST""]","[""Pramit Pal Chaudhuri""]","[""India has the "","", says a recent Harvard Uni...","[""Indian Army"",""Galwan Valley"",""India China bo...",india close militari china border,india say recent harvard univers assess chines...,opinion
2,5ef0d2bad83b3587cb500ecc,"[""opinion""]","[""Ladakh faceoff: Looking back for a way forwa...","[""DRAW THE LINE Policing the borders and preve...","[""Updated: Jun 22, 2020 18:06 IST""]","[""Lt Gen GS Sihota (retd)""]","[""On December 11, 1917, General Edmund Allenby...",NaN,ladakh faceoff look forward,decemb general edmund allenbi enter holi citi ...,opinion
3,5ef0d2bdd83b3587cb500ecd,"[""editorials""]","[""Redesign the model village scheme | HT Edito...","[""For MPs to take it up, expand its ambit to a...","[""Updated: Jun 22, 2020 18:56 IST""]",NaN,"[""A study commissioned by the Union ministry o...",NaN,redesign model villag scheme editori,studi commiss union ministri rural develop saa...,editori
4,5ef0d2bed83b3587cb500ece,"[""columns""]","[""How China has overplayed its hand, writes Ba...","[""Beijing’s belligerence reflects a familiar h...","[""Updated: Jun 22, 2020 19:31 IST""]","[""Baijayant “Jay” Panda""]","[""Almost 2,500 years ago, Greek historian, Thu...","[""CHINA"",""India""]",china overplay hand write baijay panda,year greek historian thucydid write devast pel...,column


In [37]:
#Split the data into a "train" and "test". As well as select a metric in which to measure the similarity between articles. Think of the "train" set as the corpus.
# Think of the "test" set as the NYT articles that users are currently reading.

# move articles to an array
articles = df.processed_content.values

# move article section names to an array
sections = df.processed_topic.values

# move article web_urls to an array
headings = df.processed_heading.values

# shuffle these three arrays 
articles, sections, headings = shuffle(articles, sections, headings, random_state=4)

In [38]:
# split the shuffled articles into two arrays
n = 10

# one will have all but the last 10 articles -- think of this as your training set/corpus 
X_train = articles[:-n]
X_train_headings = headings[:-n]
X_train_sections = sections[:-n]

# the other will have those last 10 articles -- think of this as your test set/corpus 
X_test = articles[-n:]
X_test_headings = headings[-n:]
X_test_sections = sections[-n:]

In [39]:
# instantiate your vectorizor 
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [40]:
# fit the vectorizer 
tfidf_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [41]:
# transform both article splits 
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [42]:
#Builidng a Content Based Recommender
def get_top_n_rec_articles(X_train_tfidf, X_train, test_article, X_train_sections, X_train_urls, n = 5):
    '''This function calculates similarity scores bewteen a document and a corpus
    
       INPUT: vectorized document corpus, 2D array
              text document corpus, 1D array
              user article, 1D array
              article section names, 1D array
              article URLs, 1D array
              number of articles to recommend, int
              
       OUTPUT: top n recommendations, 1D array
               top n corresponding section names, 1D array
               top n corresponding URLs, 1D array
               similarity scores bewteen user article and entire corpus, 1D array
              '''
    # calculate similarity between the corpus (i.e. the "test" data) and the user's article
    similarity_scores = X_train_tfidf.dot(test_article.toarray().T)

    # get sorted similairty score indicies 
    sorted_indicies = np.argsort(similarity_scores, axis = 0)[::-1]

    # get sorted similarity socres
    sorted_sim_scores = similarity_scores[sorted_indicies]

    # get top n most similar documents
    top_n_recs = X_train[sorted_indicies[:n]]

    # get top n corresponding document section names
    rec_sections = X_train_sections[sorted_indicies[:n]]

    # get top n corresponding urls
    rec_urls = X_train_urls[sorted_indicies[:n]]
    
    # return recommendations and corresponding article meta-data
    return top_n_recs, rec_sections, rec_urls, sorted_sim_scores

In [43]:
# pick an article from the "test" set
# treat this as the article that the user is currently reading
k = 0
test_article = X_test_tfidf[k]

In [51]:
# return the top n most similar articles as recommendations 
top_n_recs, rec_sections, rec_headings, sorted_sim_scores = \
get_top_n_rec_articles(X_train_tfidf, X_train,  test_article,X_train_sections, X_train_headings, n = 5 )

In [46]:
#validate the results
#Compare the user's article and corresponding section name with the recommended articles and corresponding section names

# similarity scores
sorted_sim_scores[:5]

array([[[0.17905536]],

       [[0.14565555]],

       [[0.10732024]],

       [[0.0866299 ]],

       [[0.08533849]]])

In [47]:
# user's article
X_test[k]

'govern constitut nation complianc platform dope sport chairmanship sport secretari line intern convent adopt unesco general confer panel indian olymp associ presid nation anti dope agenc director general member nada member secretari member panel member includ repres ministri home affair financ extern affair human resourc develop health justic offic memorandum date june sport ministri request ministri depart nomin repres india signatori intern convent dope sport adopt unanim unesco general confer purpos convent promot prevent fight dope sport view elimin order code compliant intern convent decid approv compet author constitut nation complianc platform dope sport chairmanship secretari sport compris senior offic ministri depart offic memorandum say'

In [48]:
# user's article's section name
X_test_sections[k]

'sport'

In [49]:
# corresponding section names for top n recs 
rec_sections

array([['sport'],
       ['india'],
       ['india'],
       ['analysi'],
       ['fit']], dtype=object)

In [50]:
# top n article recs
top_n_recs

array([['sport ministri thursday extend contract foreign coach till septemb year say futur hire year cycl sync olymp game ensur continu year contract give coach basi recommend respect nation sport feder extens current bunch hire coach keep mind postpon olymp year covid pandem contract review annual extend basi overal perform indic achiev athlet major intern event coach backbon sport ecosystem countri ensur right coach elit athlet vital step improv india chanc major intern tournament includ olymp sport minist kiren rijiju say statement decis india long term roadmap olymp prepar confid athlet benefit add indian olymp associ presid narind batra welcom decis say help athlet especi time forc break coronavirus forc lockdown recent meet sport minist repres nsfs rais issu request longer contract period foreign coach exist coach know athlet build batra say frequent chang coach mean athlet adjust tempera coach vice versa hamper level perform batra say decis posit impact perform indian athlet asi

In [53]:
# corresonding URLs for top n recs 
rec_headings

array([['ministri extend contract exist foreign coach sept coach tenur'],
       ['indian mission staffer detain pakistani agenc return wagah'],
       ['regim undergo major reform year'],
       ['mistak chines calcul analysi'],
       ['shilpa shetti kundra share secret fit easi']], dtype=object)